In [1]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import time
import urllib.parse
import numpy as np
import pandas as pd

@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

def scrape_suumo(url, max_page):
    data_samples = []
    times = []

    start = time.time()

    for page in range(1, max_page + 1):
        before = time.time()

        soup = load_page(url.format(page))
        building_list = soup.find_all(class_='cassetteitem')
        
        for building in building_list:
            building_data = extract_building_data(building)
            data_samples += extract_room_data(building, building_data, url)
        
        time.sleep(1)
        after = time.time()

        print_progress(page, max_page, before, after, data_samples, times)

    finish = time.time()
    print('総経過時間：', finish - start)

    return data_samples

def extract_building_data(building):
    building_info = []
    building_info.append(building.find(class_='ui-pct ui-pct--util1').text)
    building_info.append(building.find(class_='cassetteitem_content-title').text)
    building_info.append(building.find(class_='cassetteitem_detail-col1').text)

    details = building.find(class_='cassetteitem_detail-col2')
    for detail in details.find_all(class_='cassetteitem_detail-text'):
        building_info.append(detail.text)

    details = building.find(class_='cassetteitem_detail-col3')
    for detail in details.find_all('div'):
        building_info.append(detail.text)

    return building_info

def extract_room_data(building, building_info, url):
    room_data_samples = []
    rooms = building.find(class_='cassetteitem_other')

    for room in rooms.find_all(class_='js-cassette_link'):
        room_data = extract_single_room_data(room, url)
        room_data_samples.append(building_info + room_data)

    return room_data_samples

def extract_single_room_data(room, url):
    room_info = []

    for id_, detail in enumerate(room.find_all('td')):
        if id_ == 2:
            room_info.append(detail.text.strip())
        elif id_ == 3:
            room_info.append(detail.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
            room_info.append(detail.find(class_='cassetteitem_price cassetteitem_price--administration').text)
        elif id_ == 4:
            room_info.append(detail.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
            room_info.append(detail.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
        elif id_ == 5:
            room_info.append(detail.find(class_='cassetteitem_madori').text)
            room_info.append(detail.find(class_='cassetteitem_menseki').text)
        elif id_ == 8:
            get_url = detail.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
            abs_url = urllib.parse.urljoin(url, get_url)
            room_info.append(abs_url)

    return room_info

def print_progress(page, max_page, before, after, data_samples, times):
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    print(f'総取得件数：{len(data_samples)}')
    complete_ratio = round(page / max_page * 100, 3)
    print(f'完了：{complete_ratio}%')
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    print(f'残り時間：{int(running_required_time / 3600)}時間{int((running_required_time % 3600) / 60)}分{int(running_required_time % 60)}秒\n')


# スクレイピング実行
url = 'https://suumo.jp/chintai/kanagawa/sc_yokohamashitsurumi/?page={}'
max_page = 155
data_samples = scrape_suumo(url, max_page)

# Pandas DataFrameに変換
correct_column_names = ['カテゴリ', '建物名', '住所', 'アクセス1', 'アクセス2', 'アクセス3', '築年数', '構造', '階数', '家賃', '管理費', '敷金', '礼金', '間取り', '面積', 'URL']
df = pd.DataFrame(data_samples, columns=correct_column_names)

# CSVファイルに保存
df.to_csv('suumo_scraping.csv', index=False, encoding='utf-8-sig')


1ページ目：2.301480770111084秒
総取得件数：47
完了：0.645%
残り時間：0時間5分54秒

2ページ目：2.181043863296509秒
総取得件数：85
完了：1.29%
残り時間：0時間5分42秒

3ページ目：2.553748846054077秒
総取得件数：187
完了：1.935%
残り時間：0時間5分56秒

4ページ目：2.8620171546936035秒
総取得件数：328
完了：2.581%
残り時間：0時間6分13秒

5ページ目：2.296259641647339秒
総取得件数：364
完了：3.226%
残り時間：0時間6分5秒

6ページ目：2.250169038772583秒
総取得件数：408
完了：3.871%
残り時間：0時間5分58秒

7ページ目：2.3205556869506836秒
総取得件数：480
完了：4.516%
残り時間：0時間5分54秒

8ページ目：2.2172646522521973秒
総取得件数：521
完了：5.161%
残り時間：0時間5分48秒

9ページ目：2.213287830352783秒
総取得件数：573
完了：5.806%
残り時間：0時間5分43秒

10ページ目：2.236523151397705秒
総取得件数：616
完了：6.452%
残り時間：0時間5分39秒

11ページ目：2.153341770172119秒
総取得件数：668
完了：7.097%
残り時間：0時間5分34秒

12ページ目：2.1184463500976562秒
総取得件数：704
完了：7.742%
残り時間：0時間5分30秒

13ページ目：2.2970614433288574秒
総取得件数：747
完了：8.387%
残り時間：0時間5分27秒

14ページ目：2.2694435119628906秒
総取得件数：809
完了：9.032%
残り時間：0時間5分25秒

15ページ目：2.1703145503997803秒
総取得件数：848
完了：9.677%
残り時間：0時間5分21秒

16ページ目：2.142388105392456秒
総取得件数：883
完了：10.323%
残り時間：0時間5分17秒

17ページ目：2.1542563438415527秒
総取